In [25]:
import sys
import os
sys.path.append('..')
sys.path.append('../src')
sys.path.append('../data')
# from Src.config import hf_access_token, hf_model_cache_dir # noqa: E402
# os.environ["HF_HOME"] = hf_/model_cache_dir 
# import torch

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Load the model "gpt2" or "EleutherAI/pythia-6.9b"

In [26]:
from model import ModelFactory
model = ModelFactory.create("gpt2")

 Cuda is not available, switching to cpu 
Loaded pretrained model gpt2 into HookedTransformer


Then load the dataset (Warning, select the right dataset for the model you loaded). Load also the ablator class


In [4]:
from dataset import BaseDataset
from experiment import Ablator


dataset = BaseDataset(path = "../data/full_data_sampled_gpt2.json",
                      model = model,
                      experiment="copyVSfact",
                      no_subject=True)
ablator = Ablator(model=model, dataset=dataset, experiment="copyVSfact", batch_size=20)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 No subject found in the dataset , proceeding with no subject data


Tokenizing and computing lengths:   0%|          | 0/10000 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Tokenizing and computing lengths: 100%|██████████| 10000/10000 [00:14<00:00, 697.72it/s]


Set the heads that you want to modify, the value to multiply the heads and the token position that you want to modify (all, or attribute):

In [7]:
ablator.set_heads(heads=[(10,7), (11,10)], value=5, position="attribute")


Run the attention modification:


- mem --> logit of the factual token


- cp --> logit of the counterfactual token

- mem win --> number of factual predictions

- cp win --> number of counterfactual predictions

In [8]:
ablator.run()

Ablating: 100%|██████████| 24/24 [01:11<00:00,  2.96s/it]


,mem,cp,diff,mem_std,cp_std,diff_std,mem_win,cp_win
0,13.857364,14.130378,-0.273015,1.666047,2.702236,2.729101,5014.0,4614.0


In [8]:
# grid search for alpha/multiplier
result = []
for alpha in [2,5,10,100]:
    ablator.set_heads(heads=[(10,7), (11,10)], value=alpha, position="attribute")
    result.append(ablator.run())

Ablating:   0%|          | 0/24 [00:00<?, ?it/s]

Ablating: 100%|██████████| 24/24 [00:35<00:00,  1.47s/it]


In [9]:
result

[         mem         cp      diff   mem_std    cp_std  diff_std  mem_win  \
 0  13.947168  15.336194 -1.389027  1.637497  2.689528  2.649626   3217.0   
 
    cp_win  
 0  6554.0  ,
          mem         cp      diff   mem_std    cp_std  diff_std  mem_win  \
 0  13.857364  14.130377 -0.273015  1.666047  2.702235  2.729101   5014.0   
 
    cp_win  
 0  4614.0  ,
          mem         cp      diff   mem_std    cp_std  diff_std  mem_win  \
 0  13.839895  13.989985 -0.150091  1.663305  2.688327  2.742588   5156.0   
 
    cp_win  
 0  4456.0  ,
          mem         cp      diff   mem_std    cp_std  diff_std  mem_win  \
 0  13.831625  13.926047 -0.094424  1.660842  2.670364  2.741903   5229.0   
 
    cp_win  
 0  4374.0  ]

## Use your own modification

You can pass to the model your own modification function. You can pass a list of hook (string, function) to the model using set_hooks method. The hooks should be consistent with the hook of the transformer lens library (https://neelnanda-io.github.io/TransformerLens/index.html)

In [ ]:
from dataset import BaseDataset
from experiment import Ablator


dataset = BaseDataset(path = "../data/cft_og_combined_data_sampled_gpt2_with_questions.json",
                      model = model,
                      experiment="copyVSfactQnA",
                      prompt_type="qna",
                      no_subject=True)
ablator = Ablator(model=model, dataset=dataset, experiment="copyVSfactQnA", batch_size=20)

In [29]:
ablator.set_heads(heads=[(11,3)], value=5, position="attribute")
result = ablator.run()
result

Ablating: 100%|██████████| 29/29 [01:22<00:00,  2.84s/it]


,mem,cp,diff,mem_std,cp_std,diff_std,mem_win,cp_win
0,12.591045,13.703269,-1.112226,2.140498,1.049417,2.481231,3702.0,6359.0


In [30]:
ablator.set_heads(heads=[(10,7)], value=5, position="attribute")
result = ablator.run()
result

Ablating: 100%|██████████| 29/29 [02:49<00:00,  5.84s/it]


,mem,cp,diff,mem_std,cp_std,diff_std,mem_win,cp_win
0,12.800153,12.35463,0.445524,2.290849,1.292336,2.723206,5929.0,1916.0


In [44]:
ablator.set_heads(heads=[(10,7)], value=5, position="interrogative")
result = ablator.run()
result

Ablating: 100%|██████████| 29/29 [01:52<00:00,  3.87s/it]


,mem,cp,diff,mem_std,cp_std,diff_std,mem_win,cp_win
0,12.68951,13.73616,-1.04665,2.189614,1.018348,2.490377,3886.0,6246.0


In [48]:
ablator.set_heads(heads=[(11,3)], value=5, position="interrogative")
result = ablator.run()
result

Ablating: 100%|██████████| 29/29 [03:47<00:00,  7.84s/it]


,mem,cp,diff,mem_std,cp_std,diff_std,mem_win,cp_win
0,12.606985,13.596092,-0.989104,2.143174,1.017077,2.453731,3980.0,6184.0


In [31]:
dataset.full_data[0]

{'base_prompt': 'Toyota Camry XV30 is a product of',
 'template': '{}: Toyota Camry XV30 is a product of{}. Toyota Camry XV30 is a product of',
 'target_true': ' Toyota',
 'target_new': ' Chrysler',
 'prompt': 'Redefine: Toyota Camry XV30 is a product of Chrysler. What company is the Toyota Camry XV30 a product of? Answer:',
 'subject': 'Toyota Camry XV30',
 'question': 'What company is the Toyota Camry XV30 a product of?',
 'idx': 'og_0',
 'tokenized_prompt': tensor([ 7738,   891,   500,    25, 20182,  7298,   563, 29922,  1270,   318,
           257,  1720,   286, 36477,    13,  1867,  1664,   318,   262, 20182,
          7298,   563, 29922,  1270,   257,  1720,   286,    30, 23998,    25]),
 'target_new_token': tensor([36477]),
 'target_true_token': tensor([20182]),
 'targets': tensor([20182, 36477]),
 '1_subj_pos': -1,
 '2_subj_pos': -1,
 'subj_len': -1,
 'obj_pos': 13,
 'length': 30}

In [42]:
obj_pos = 13
tokenizer.decode(dataset.full_data[0]["tokenized_prompt"][obj_pos+2])

' What'

In [47]:
tokenizer.decode(23998)

' Answer'